*If you run this notebook at colab.research.google.com, you need to install packages with the following command*:

In [ ]:
# !pip install -r https://raw.githubusercontent.com/obidam/ds2-2022/main/requirements.txt
# !curl https://raw.githubusercontent.com/obidam/ds2-2022/main/practice/exploratory_statitics/tuto_tools.py --output tuto_tools.py

# Clustering - Tuto 0

In this 1st tuto, we simply play with KMeans and GMM on a toy multimodal dataset

The objective is to illustrate a "clustering" method

In [ ]:
# Libraries import section
import os

import xarray as xr
import numpy as np

from tuto_tools import new_sample, translate, rotate, stretch, plot2d_labels, plot_GMMellipse, sns_plot2d_GMM_marginals

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
%matplotlib inline

import seaborn as sns
sns.set(context="notebook", style="whitegrid", palette="deep", color_codes=True)

import pandas as pd

# Create a dummy dataset

we use a 2D sample datasets with Normal distributions

The dataset array is in "X", true labels in "y"


In [ ]:
n = 500 # Number of samples per cluster

# Create the 2st cluster:
il = 0 # Init cluster IDs
s = new_sample([10,0],1,n=n)
X, y = s['data'], s['labels'] # 1st label is "0"

# Add another one:
il += 1
X, y = np.concatenate((X,new_sample([5,0],2,n=n)['data']),axis=0), np.concatenate((y,np.full((n),il)),axis=0)

# Add another one:
il += 1
X, y = np.concatenate((X,new_sample([5,5],0.7,n=n)['data']),axis=0), np.concatenate((y,np.full((n),il)),axis=0)

# Add another one:
il += 1
x = translate(rotate(stretch(new_sample([0,0],1,n=2*n)['data'],m=[5,1]),a=45),v=[0,5])
X, y = np.concatenate((X,x),axis=0), np.concatenate((y,np.full((2*n),il)),axis=0)

# Final nb of clusters:
n_clusters_true = il+1
print("Create a dummy dataset with %i clusters"%n_clusters_true)

In [ ]:
# Plot the dummy dataset
plt.figure(figsize=(10,10), dpi=160, facecolor='w', edgecolor='k')
plt.subplot(221)
plt.plot(X[:, 0], X[:, 1],'k.',markersize=5)
plt.axis('equal')
plt.title("Raw dataset")
plt.subplot(222)
plt.scatter(X[:, 0], X[:, 1], 5, c=y, cmap=ListedColormap(sns.husl_palette(n_clusters_true).as_hex()))
plt.axis('equal')
plt.title("True %i cluster labels"%n_clusters_true);

# Clustering with KMeans

To see how KMeans iterates to identify clusters, see the Stanford interactive tool:

http://stanford.edu/class/ee103/visualizations/kmeans/kmeans.html

In [ ]:
from sklearn.cluster import KMeans

# Create the classifier:
kmeans = KMeans(n_clusters=n_clusters_true, random_state=0)

# Train the classifier:
kmeans.fit(X)

# Classify the dataset:
labels = kmeans.predict(X)

#
n_clusters_ = kmeans.n_clusters
print("Cluster centers shape:",kmeans.cluster_centers_.shape)
print("Cluster centers:\n",kmeans.cluster_centers_)

In [ ]:
# Plot labels of data with cluster centers:
fig, ax, colors = plot2d_labels(X,labels,cluster_centers=kmeans.cluster_centers_,kmarkersize=14)

# Clustering with GMM

In [ ]:
from sklearn.mixture import GaussianMixture as GMM

# Create classifier:
gmm = GMM(n_components=n_clusters_true, random_state=0)

# Train it:
gmm.fit(X)

# Classify data:
labels = gmm.predict(X)

# 
n_clusters_ = gmm.n_components
print("Cluster centers shape:",gmm.means_.shape)
print("Cluster centers:\n",gmm.means_)

In [ ]:
# Plot labels of data with cluster centers:
fig, ax, colors = plot2d_labels(X,labels,cluster_centers=gmm.means_,kmarkersize=14)

## Visualise Gaussian Modes of each clusters

In [ ]:
# Simple plot with labelled data:
fig, ax, colors = plot2d_labels(X, labels, dpi=80)

# Super-impose the GMM modes:
for k, col in zip(range(n_clusters_), colors):
    el,ax = plot_GMMellipse(gmm,[0,1],k,col,ax,std=[1],label="Class-%i Gaussian"%(k),linewidth=4)
    el,ax = plot_GMMellipse(gmm,[0,1],k,'k',ax,std=[1],linewidth=2)
    el,ax = plot_GMMellipse(gmm,[0,1],k,'k',ax,std=[2],linewidth=1)
#     id = [0,1]
#     covariances = gmm.covariances_[k][(id[0],id[0],id[1],id[1]),(id[0],id[1],id[0],id[1])].reshape(2,2)
#     print "Cluster %i covariance matrix:\n"%k, covariances
#     print "Cluster %i center matrix:\n"%k, gmm.means_[k,:]

ax.axis('equal')
plt.title('Number of clusters: %i (GMM)' % n_clusters_)
plt.legend()
plt.show()

## Super-imposed Gaussian Modes with the dataset pdf

In [ ]:
# Assemble variables to use in a Pandas dataframe:
df = pd.DataFrame(np.concatenate((X,labels[np.newaxis].T),axis=1), columns=["x", "y", "labels"])

# Plot the 2d data, with GMM modes and marginal distributions:
g = sns_plot2d_GMM_marginals(df, gmm)

## Exercise

Modify the imposed shape of the covariance matrix in the GMM and visualise new results

The option name is `covariance_type` and possible values are `['spherical', 'diag', 'tied', 'full']`

In [ ]:
from sklearn.mixture import GaussianMixture as GMM